In [1]:
from current_season_code.functions import *
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import jupyterthemes as jt

In [2]:
bets = pd.read_csv('current_season_data/bets.csv', index_col = 0)

In [3]:
bets[['Team1', 'Team2', 'Team1_Prob', 'Team2_Prob', 'Team1_KC', 'Team2_KC', 'Team1_Bet', 'Team2_Bet']]

,Team1,Team2,Team1_Prob,Team2_Prob,Team1_KC,Team2_KC,Team1_Bet,Team2_Bet
0,Pacers,Mavericks,0.644346,0.355654,0.018723,0,1509.272431,0
1,Hornets,Kings,0.697017,0.302983,0.028796,0,2321.310279,0
2,Hawks,Nets,0.588223,0.411777,0.004409,0,0.000000,0
3,Raptors,Knicks,0.611970,0.388030,0.012841,0,1035.149651,0
4,Rockets,Bucks,0.436375,0.563625,0.021213,0,1710.026314,0
5,Thunder,Lakers,0.589238,0.410762,0.033182,0,2674.840887,0
6,Timberwolves,Cavaliers,0.538434,0.461566,0.000251,0,0.000000,0
7,Pelicans,Pistons,0.744088,0.255912,0.012958,0,1044.527906,0
8,Suns,Celtics,0.710019,0.289981,0.001172,0,0.000000,0


In [7]:
bets_external = pd.read_csv('current_season_data/bets_external.csv', index_col = 0)
bets_external[['Away_Team', 'Home_Team', 'Home_Bet', 'Away_Bet', 'Home_Bet_Combined', 'Away_Bet_Combined']]

,Away_Team,Home_Team,Home_Bet,Away_Bet,Home_Bet_Combined,Away_Bet_Combined
0,Mavericks,Pacers,3542.300099,0,3263.714918,0
1,Kings,Hornets,1171.509085,0,2831.475788,0
2,Nets,Hawks,2171.331493,0,1452.769163,0
3,Knicks,Raptors,2441.485528,0,2244.572933,0
4,Cavaliers,Timberwolves,3352.939794,0,1735.076730,0
5,Pistons,Pelicans,4292.166905,0,3200.521626,0
6,Lakers,Thunder,2081.495442,0,3637.022117,0
7,Bucks,Rockets,602.695163,0,1952.653294,0
8,Celtics,Suns,1893.603046,0,0.000000,0


In [10]:
# Getting NBA injuries

tables = pd.read_html('https://www.cbssports.com/nba/injuries/')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.cbssports.com/nba/injuries/')
team_names = driver.find_elements_by_xpath('//*[@id="TableBase"]/h4')
table_dict = {}
for i in range(len(tables)):
    table_dict[team_names[i].text] = tables[i]
for key,value in table_dict.items():
    value['Player'] = value.Player.str.split()
    for index, row in value.iterrows():
        if len(row.Player) == 4:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2]
        if len(row.Player) == 6:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2] + ' ' + row.Player[-3]
for key,value in table_dict.items():
    value['Player'] = value.Player.str.split()
    for index, row in value.iterrows():
        if len(row.Player) == 2:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2]
        if len(row.Player) == 3:
            value.loc[index, 'Player'] = row.Player[-1] + ' ' + row.Player[-2] + ' ' + row.Player[-3]



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [11]:
# Getting NBA stats

